# Análise de Retenção por Cohort - Olist E-Commerce

**Autor:** André Bomfim  
**Data:** Novembro 2024  
**Objetivo:** Analisar retenção de clientes ao longo do tempo usando cohort analysis para identificar padrões de churn e oportunidades de fidelização

---

## 📋 Índice

1. [Setup e Configuração](#1-setup)
2. [Cohort Base e Matriz de Retenção](#2-matriz)
3. [Curvas de Retenção](#3-curvas)
4. [Análise de Churn](#4-churn)
5. [Retenção por Estado](#5-estado)
6. [Benchmark e Curva Média](#6-benchmark)
7. [Métricas Agregadas por Cohort](#7-metricas)
8. [Insights e Conclusões](#8-insights)

## 1. Setup e Configuração

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv
import os

warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuração do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Carregar variáveis de ambiente
load_dotenv()

# Configuração BigQuery
PROJECT_ID = os.getenv('GCP_PROJECT_ID')
DATASET_ID = os.getenv('GCP_DATASET_ID', 'olist_ecommerce')

# Cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Criar diretório para imagens
os.makedirs('../docs/images', exist_ok=True)

print(f"Setup completo - Projeto: {PROJECT_ID}, Dataset: {DATASET_ID}")

In [ ]:
def query_bigquery(query: str) -> pd.DataFrame:
    """Helper para executar queries no BigQuery"""
    return client.query(query).to_dataframe()

def save_plot(fig, filename):
    """Salvar gráfico com tratamento de erro"""
    try:
        fig.savefig(f'../docs/images/{filename}', dpi=300, bbox_inches='tight')
        print(f"✓ Gráfico salvo: docs/images/{filename}")
    except Exception as e:
        print(f"⚠️ Erro ao salvar gráfico: {e}")

## 2. Cohort Base e Matriz de Retenção

In [ ]:
# Query Matriz de Retenção
query_retention_matrix = f"""
WITH cohort_base AS (
    -- Primeira compra de cada cliente
    SELECT 
        c.customer_unique_id,
        DATE_TRUNC(MIN(o.order_purchase_timestamp), MONTH) as cohort_month
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_unique_id
),
customer_activity AS (
    -- Todas compras com cohort_month
    SELECT 
        cb.customer_unique_id,
        cb.cohort_month,
        DATE_TRUNC(o.order_purchase_timestamp, MONTH) as activity_month,
        DATE_DIFF(DATE_TRUNC(o.order_purchase_timestamp, MONTH), cb.cohort_month, MONTH) as months_since_first_purchase,
        p.payment_value
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
    JOIN cohort_base cb ON c.customer_unique_id = cb.customer_unique_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.payments` p ON o.order_id = p.order_id
    WHERE o.order_status = 'delivered'
),
retention_data AS (
    -- Matriz de retenção
    SELECT
        cohort_month,
        months_since_first_purchase,
        COUNT(DISTINCT customer_unique_id) as active_customers,
        SUM(payment_value) as revenue
    FROM customer_activity
    WHERE months_since_first_purchase BETWEEN 0 AND 12
    GROUP BY cohort_month, months_since_first_purchase
),
cohort_sizes AS (
    -- Tamanho inicial dos cohorts
    SELECT
        cohort_month,
        COUNT(DISTINCT customer_unique_id) as cohort_size
    FROM cohort_base
    GROUP BY cohort_month
)
SELECT
    rd.cohort_month,
    rd.months_since_first_purchase,
    rd.active_customers,
    rd.revenue,
    cs.cohort_size,
    (rd.active_customers / cs.cohort_size) * 100 as retention_rate_pct
FROM retention_data rd
JOIN cohort_sizes cs ON rd.cohort_month = cs.cohort_month
WHERE cs.cohort_size >= 50  -- Filtrar cohorts pequenos
ORDER BY rd.cohort_month, rd.months_since_first_purchase
"""

df_retention = query_bigquery(query_retention_matrix)
print(f"✓ {len(df_retention):,} registros de retenção carregados")
print(f"✓ Cohorts analisados: {df_retention['cohort_month'].nunique()}")
print(f"✓ Período: {df_retention['cohort_month'].min()} a {df_retention['cohort_month'].max()}")

In [ ]:
# Preparar matriz de retenção (pivot table)
df_pivot = df_retention.pivot_table(
    index='cohort_month',
    columns='months_since_first_purchase',
    values='retention_rate_pct',
    aggfunc='first'
).fillna(0)

# Ordenar cohorts cronologicamente
df_pivot = df_pivot.sort_index()

# Renomear colunas
df_pivot.columns = [f'M{int(col)}' for col in df_pivot.columns]

print("MATRIZ DE RETENÇÃO (Primeiras 5 cohorts):")
print(df_pivot.head())

# Estatísticas básicas
print(f"\nESTATÍSTICAS GERAIS:")
print(f"  • Cohorts analisados: {len(df_pivot)}")
print(f"  • Retenção M1 média: {df_pivot['M1'].mean():.2f}%")
print(f"  • Retenção M3 média: {df_pivot['M3'].mean():.2f}%")
print(f"  • Retenção M6 média: {df_pivot['M6'].mean():.2f}%")

In [ ]:
# Heatmap da Matriz de Retenção
plt.close('all')
fig, ax = plt.subplots(figsize=(16, 10))

# Criar heatmap
sns.heatmap(
    df_pivot,
    annot=True,
    fmt='.1f',
    cmap='RdYlGn',
    center=5,
    ax=ax,
    cbar_kws={'label': 'Taxa de Retenção (%)'},
    linewidths=0.5,
    annot_kws={'size': 9}
)

ax.set_title('Matriz de Retenção - Cohorts por Mês (Retenção %)', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Meses desde Primeira Compra', fontsize=12)
ax.set_ylabel('Cohort (Mês Primeira Compra)', fontsize=12)

plt.tight_layout()
save_plot(fig, 'cohort_retention_matrix.png')
plt.show()

## 3. Curvas de Retenção

In [ ]:
# Preparar dados para curvas de retenção
df_curves = df_retention.copy()

# Calcular métricas agregadas
retention_aggregated = df_curves.groupby('months_since_first_purchase').agg({
    'retention_rate_pct': ['mean', 'std', 'median', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]
}).round(2)

retention_aggregated.columns = ['mean', 'std', 'median', 'p25', 'p75']
retention_aggregated = retention_aggregated.reset_index()

print(" CURVA MÉDIA DE RETENÇÃO:")
print(retention_aggregated.head(8))

In [ ]:
# Visualização Curvas de Retenção
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Curvas individuais dos últimos 6 cohorts
recent_cohorts = df_pivot.tail(6)
for cohort in recent_cohorts.index:
    axes[0, 0].plot(
        range(len(recent_cohorts.columns)),
        recent_cohorts.loc[cohort].values,
        marker='o',
        linewidth=2,
        alpha=0.7,
        label=cohort.strftime('%Y-%m')
    )
axes[0, 0].set_xlabel('Meses desde Primeira Compra')
axes[0, 0].set_ylabel('Taxa de Retenção (%)')
axes[0, 0].set_title('Curvas de Retenção - Últimos 6 Cohorts')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Gráfico 2: Curva média com intervalo de confiança
months = retention_aggregated['months_since_first_purchase']
mean_retention = retention_aggregated['mean']
p25_retention = retention_aggregated['p25']
p75_retention = retention_aggregated['p75']

axes[0, 1].plot(months, mean_retention, linewidth=3, marker='o', label='Média', color='blue')
axes[0, 1].fill_between(months, p25_retention, p75_retention, alpha=0.3, color='blue', label='P25-P75')
axes[0, 1].set_xlabel('Meses desde Primeira Compra')
axes[0, 1].set_ylabel('Taxa de Retenção (%)')
axes[0, 1].set_title('Curva Média de Retenção com Intervalo P25-P75')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Gráfico 3: Comparação M1, M3, M6 retention
key_months = ['M1', 'M3', 'M6']
key_retention = [df_pivot['M1'].mean(), df_pivot['M3'].mean(), df_pivot['M6'].mean()]

bars = axes[1, 0].bar(key_months, key_retention, color=['red', 'orange', 'green'], alpha=0.7)
axes[1, 0].set_ylabel('Taxa de Retenção (%)')
axes[1, 0].set_title('Retenção em Momentos Chave (M1, M3, M6)')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for bar, value in zip(bars, key_retention):
    axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                   f'{value:.2f}%', ha='center', va='bottom', fontweight='bold')

# Gráfico 4: Churn incremental
churn_incremental = []
for i in range(1, len(mean_retention)):
    churn = mean_retention.iloc[i-1] - mean_retention.iloc[i]
    churn_incremental.append(churn)

axes[1, 1].bar(range(1, len(mean_retention)), churn_incremental, 
               color='red', alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Mês (Churn Mês N-1 → Mês N)')
axes[1, 1].set_ylabel('Churn Incremental (% pontos)')
axes[1, 1].set_title('Churn Incremental por Mês')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
save_plot(fig, 'cohort_retention_curves.png')
plt.show()

## 4. Análise de Churn 

In [ ]:
# Calcular análise de churn a partir dos dados existentes
df_churn = df_retention.groupby('months_since_first_purchase').agg({
    'retention_rate_pct': ['mean', 'std'],
    'cohort_month': 'nunique'
}).reset_index()

df_churn.columns = ['months_since_first_purchase', 'avg_retention_rate', 'std_retention_rate', 'cohort_count']
df_churn['avg_churn_rate'] = 100 - df_churn['avg_retention_rate']

print(" ANÁLISE DE CHURN:")
print(df_churn.head(8))

In [ ]:
# Visualização Análise de Churn
plt.close('all')
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: Evolução do Churn Rate
months = df_churn['months_since_first_purchase']
churn_rates = df_churn['avg_churn_rate']

axes[0].plot(months, churn_rates, linewidth=3, marker='o', color='red', label='Churn Rate')
axes[0].fill_between(months, 
                    churn_rates - df_churn['std_retention_rate'], 
                    churn_rates + df_churn['std_retention_rate'], 
                    alpha=0.3, color='red', label='±1 Std Dev')

axes[0].set_xlabel('Meses desde Primeira Compra')
axes[0].set_ylabel('Taxa de Churn Acumulada (%)')
axes[0].set_title('Evolução do Churn Rate ao Longo do Tempo')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Destacar churn crítico M0→M1
critical_churn = churn_rates.iloc[1] - churn_rates.iloc[0]
axes[0].annotate(f'Churn Crítico M0→M1: {critical_churn:.1f}%',
                xy=(0.5, churn_rates.iloc[1]), xytext=(2, churn_rates.iloc[1] + 10),
                arrowprops=dict(arrowstyle='->', color='darkred'),
                fontweight='bold', color='darkred')

# Gráfico 2: Churn Incremental (Waterfall)
incremental_churn = []
for i in range(1, len(churn_rates)):
    incremental = churn_rates.iloc[i] - churn_rates.iloc[i-1]
    incremental_churn.append(incremental)

# Criar waterfall
positions = range(1, len(churn_rates))
colors = ['red' if churn > 5 else 'orange' if churn > 2 else 'yellow' for churn in incremental_churn]

bars = axes[1].bar(positions, incremental_churn, color=colors, alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Mês (Churn Mês N-1 → Mês N)')
axes[1].set_ylabel('Churn Incremental (% pontos)')
axes[1].set_title('Churn Incremental por Mês (Waterfall)')
axes[1].set_xticks(positions)
axes[1].set_xticklabels([f'M{i-1}→M{i}' for i in positions])
axes[1].grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for bar, value in zip(bars, incremental_churn):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                f'{value:.1f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
save_plot(fig, 'cohort_churn_analysis.png')
plt.show()

## 5. Retenção por Estado 

In [ ]:
# Query Corrigida - Retenção por Estado
query_retention_state = f"""
WITH cohort_base AS (
    -- Primeira compra de cada cliente com estado
    SELECT 
        c.customer_unique_id,
        c.customer_state,
        DATE_TRUNC(MIN(o.order_purchase_timestamp), MONTH) as cohort_month
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_unique_id, c.customer_state
),
all_purchases AS (
    -- Todas as compras dos clientes
    SELECT 
        c.customer_unique_id,
        DATE_TRUNC(o.order_purchase_timestamp, MONTH) as purchase_month
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
    WHERE o.order_status = 'delivered'
)
SELECT
    cb.customer_state,
    COUNT(DISTINCT cb.customer_unique_id) as total_customers,
    COUNT(DISTINCT CASE 
        WHEN DATE_DIFF(ap.purchase_month, cb.cohort_month, MONTH) = 1 
        THEN cb.customer_unique_id 
    END) * 100.0 / COUNT(DISTINCT cb.customer_unique_id) as m1_retention_pct,
    COUNT(DISTINCT CASE 
        WHEN DATE_DIFF(ap.purchase_month, cb.cohort_month, MONTH) = 3 
        THEN cb.customer_unique_id 
    END) * 100.0 / COUNT(DISTINCT cb.customer_unique_id) as m3_retention_pct
FROM cohort_base cb
LEFT JOIN all_purchases ap ON cb.customer_unique_id = ap.customer_unique_id
GROUP BY cb.customer_state
HAVING COUNT(DISTINCT cb.customer_unique_id) >= 100  -- Estados com volume significativo
ORDER BY m1_retention_pct DESC
"""

df_state_retention = query_bigquery(query_retention_state)
print(f"✓ Retenção calculada para {len(df_state_retention)} estados")
print("\n TOP 5 ESTADOS - RETENÇÃO M1:")
print(df_state_retention.head())

In [ ]:
# Visualização Retenção por Estado
plt.close('all')
fig, ax = plt.subplots(figsize=(12, 8))

# Top 10 estados por retenção M1
top_states = df_state_retention.head(10).sort_values('m1_retention_pct', ascending=True)

# Criar barras horizontais
y_pos = np.arange(len(top_states))
bars = ax.barh(y_pos, top_states['m1_retention_pct'], 
               color='green', alpha=0.7, edgecolor='black')

ax.set_yticks(y_pos)
ax.set_yticklabels(top_states['customer_state'])
ax.set_xlabel('Taxa de Retenção M1 (%)')
ax.set_title('Top 10 Estados - Retenção no Primeiro Mês (M1)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Adicionar valores nas barras
for i, (bar, retention, total) in enumerate(zip(bars, top_states['m1_retention_pct'], top_states['total_customers'])):
    ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
           f'{retention:.1f}% ({total:,} clientes)', 
           va='center', ha='left', fontsize=10)

plt.tight_layout()
save_plot(fig, 'cohort_retention_by_state.png')
plt.show()

## 6. Benchmark e Curva Média 

In [ ]:
# Preparar dados para benchmark a partir dos dados existentes
benchmark_data = df_retention.groupby('months_since_first_purchase').agg({
    'retention_rate_pct': ['mean', 'std', 'min', 'max', 'median']
}).round(2)

benchmark_data.columns = ['mean', 'std', 'min', 'max', 'median']
benchmark_data = benchmark_data.reset_index()

print(" BENCHMARK DE RETENÇÃO:")
print(benchmark_data.head(8))

In [ ]:
# Visualização Benchmark de Retenção
plt.close('all')
fig, ax = plt.subplots(figsize=(14, 8))

months = benchmark_data['months_since_first_purchase']
mean_retention = benchmark_data['mean']
std_retention = benchmark_data['std']

# Curva média com intervalo de confiança
ax.plot(months, mean_retention, linewidth=4, marker='o', 
        label='Retenção Média', color='blue', markersize=8)

# Área de ±1 desvio padrão
ax.fill_between(months, 
               mean_retention - std_retention, 
               mean_retention + std_retention, 
               alpha=0.3, color='blue', label='±1 Desvio Padrão')

# Área min-max
ax.fill_between(months, 
               benchmark_data['min'], 
               benchmark_data['max'], 
               alpha=0.1, color='gray', label='Range Min-Max')

ax.set_xlabel('Meses desde Primeira Compra', fontsize=12)
ax.set_ylabel('Taxa de Retenção (%)', fontsize=12)
ax.set_title('Benchmark de Retenção - Curva Média com Intervalos', 
             fontsize=16, fontweight='bold', pad=20)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Adicionar anotações para momentos chave
key_months = [1, 3, 6, 12]
for month in key_months:
    if month < len(mean_retention):
        retention = mean_retention.iloc[month]
        ax.annotate(f'M{month}: {retention:.1f}%', 
                   xy=(month, retention), 
                   xytext=(month+0.5, retention+2),
                   arrowprops=dict(arrowstyle='->', color='red'),
                   fontweight='bold', color='red')

plt.tight_layout()
save_plot(fig, 'cohort_retention_benchmark.png')
plt.show()

## 7. Métricas Agregadas por Cohort 

In [ ]:
# Calcular métricas agregadas a partir dos dados existentes
cohort_metrics_data = []
for cohort in df_retention['cohort_month'].unique():
    cohort_data = df_retention[df_retention['cohort_month'] == cohort]
    
    metrics = {
        'cohort_month': cohort,
        'cohort_size': cohort_data['cohort_size'].max(),
        'm1_retention': cohort_data[cohort_data['months_since_first_purchase'] == 1]['retention_rate_pct'].max() if 1 in cohort_data['months_since_first_purchase'].values else 0,
        'm3_retention': cohort_data[cohort_data['months_since_first_purchase'] == 3]['retention_rate_pct'].max() if 3 in cohort_data['months_since_first_purchase'].values else 0,
        'm6_retention': cohort_data[cohort_data['months_since_first_purchase'] == 6]['retention_rate_pct'].max() if 6 in cohort_data['months_since_first_purchase'].values else 0,
        'total_revenue': cohort_data['revenue'].sum(),
        'revenue_per_customer': cohort_data['revenue'].sum() / cohort_data['cohort_size'].max()
    }
    metrics['cohort_quality_score'] = (metrics['m1_retention'] * 0.5 + metrics['m3_retention'] * 0.3 + metrics['m6_retention'] * 0.2)
    cohort_metrics_data.append(metrics)

df_cohort_metrics = pd.DataFrame(cohort_metrics_data)

print(" MÉTRICAS AGRAGADAS POR COHORT:")
print(df_cohort_metrics.round(2))

# Sumário estatístico
print(f"\n SUMÁRIO ESTATÍSTICO:")
print(f"  • Cohorts analisados: {len(df_cohort_metrics)}")
print(f"  • Retenção M1 média: {df_cohort_metrics['m1_retention'].mean():.2f}%")
print(f"  • Retenção M3 média: {df_cohort_metrics['m3_retention'].mean():.2f}%")
print(f"  • Retenção M6 média: {df_cohort_metrics['m6_retention'].mean():.2f}%")
print(f"  • Revenue por cliente médio: R$ {df_cohort_metrics['revenue_per_customer'].mean():.2f}")
print(f"  • Cohort quality score médio: {df_cohort_metrics['cohort_quality_score'].mean():.2f}")

## 8. Insights e Conclusões 

In [ ]:
# Sumário Executivo Final
print("\n" + "="*80)
print(" RESUMO EXECUTIVO - ANÁLISE DE RETENÇÃO POR COHORT")
print("="*80)

# Coletar métricas finais
m1_retention_avg = df_cohort_metrics['m1_retention'].mean()
m3_retention_avg = df_cohort_metrics['m3_retention'].mean()
m6_retention_avg = df_cohort_metrics['m6_retention'].mean()
critical_churn_m0_m1 = 100 - m1_retention_avg

# Melhor e pior cohort
best_cohort = df_cohort_metrics.loc[df_cohort_metrics['cohort_quality_score'].idxmax()]
worst_cohort = df_cohort_metrics.loc[df_cohort_metrics['cohort_quality_score'].idxmin()]

# Melhor estado
best_state = df_state_retention.iloc[0]

print(f"\n MÉTRICAS CHAVE DE RETENÇÃO:")
print(f"   • Retenção M0 → M1: {m1_retention_avg:.2f}% (Churn: {critical_churn_m0_m1:.2f}%)")
print(f"   • Retenção M0 → M3: {m3_retention_avg:.2f}%")
print(f"   • Retenção M0 → M6: {m6_retention_avg:.2f}%")
print(f"   • Plateau de retenção: ~{benchmark_data[benchmark_data['months_since_first_purchase'] >= 6]['mean'].mean():.2f}% (M6+)")

print(f"\n PERFORMANCE POR COHORT:")
print(f"   • Melhor cohort: {best_cohort['cohort_month'].strftime('%Y-%m')} (Score: {best_cohort['cohort_quality_score']:.2f})")
print(f"   • Pior cohort: {worst_cohort['cohort_month'].strftime('%Y-%m')} (Score: {worst_cohort['cohort_quality_score']:.2f})")
print(f"   • Variação qualidade: {(best_cohort['cohort_quality_score'] - worst_cohort['cohort_quality_score']):.2f} pontos")

print(f"\n PERFORMANCE GEOGRÁFICA:")
print(f"   • Melhor estado: {best_state['customer_state']} (M1: {best_state['m1_retention_pct']:.2f}%)")
print(f"   • Diferença vs média: +{(best_state['m1_retention_pct'] - m1_retention_avg):.2f}%")
print(f"   • Estados analisados: {len(df_state_retention)} com volume significativo")

print(f"\n PADRÕES DE CHURN:")
print(f"   • Churn crítico M0→M1: {critical_churn_m0_m1:.2f}% (Janela de Oportunidade)")
print(f"   • Estabilização: M3+ (retenção estabiliza em ~{m3_retention_avg:.2f}%)")
print(f"   • Clientes fidelizados: {m6_retention_avg:.2f}% permanecem ativos em M6")

print(f"\n" + "="*80)
print(" RECOMENDAÇÕES ESTRATÉGICAS")
print("="*80)

print(f"\n PRIORIDADE 1: REDUZIR CHURN CRÍTICO (M0→M1)")
print(f"   • Campanha win-back: D+7 e D+30 após primeira compra")
print(f"   • Incentivo segunda compra: Cupom 15-20% válido 30 dias")
print(f"   • Meta: Aumentar retenção M1 para {m1_retention_avg + 2:.2f}% (+2%)")

print(f"\n PRIORIDADE 2: MARKETING AUTOMATIZADO")
print(f"   • Email marketing: D+7 (check-in), D+15 (oferta), D+30 (win-back)")
print(f"   • Segmentação: Foco em estados com alta retenção")
print(f"   • Personalização: Baseado em comportamento de compra")

print(f"\n PRIORIDADE 3: PROGRAMA FIDELIDADE")
print(f"   • Iniciar em M2: Clientes que superaram churn crítico")
print(f"   • Benefícios: Frete grátis, cashback, atendimento prioritário")
print(f"   • Meta: Aumentar retenção M6 para {m6_retention_avg + 1:.2f}% (+1%)")

print(f"\n PRIORIDADE 4: ANÁLISE CONTÍNUA")
print(f"   • Monitorar cohorts mensalmente")
print(f"   • A/B testing de estratégias de retenção")
print(f"   • Correlation analysis: NPS vs Retenção")

print(f"\n" + "="*80)
print(" ANÁLISE DE RETENÇÃO CONCLUÍDA")
print("="*80)